In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# DATA

In [ ]:
file_path = '../../src/data/temp/processed_zbp_detail_data.csv'
zbp_detail = pd.read_csv(file_path)
zbp_detail.head()

In [ ]:
file_path = '../../src/data/temp/processed_zbp_totals_data.csv'
zbp_totals = pd.read_csv(file_path)
zbp_totals.head()

In [ ]:
naics_mapping = {11: 'Agriculture, Forestry, Fishing and Hunting',
                 21: 'Mining',
                 22: 'Utilities',
                 23: 'Construction', 
                 31: 'Manufacturing',
                 32: 'Manufacturing',
                 33: 'Manufacturing',
                 42: 'Wholesale Trade',
                 44: 'Retail Trade',
                 45: 'Retail Trade',
                 48: 'Transportation and Warehousing',
                 49: 'Transportation and Warehousing',
                 51: 'Information',
                 52: 'Finance and Insurance',
                 53: 'Real Estate Rental and Leasing',
                 54: 'Professional, Scientific, and Technical Services',
                 55: 'Management of Companies and Enterprises',
                 56: 'Administrative and Support and Waste… Services',
                 61: 'Educational Services',
                 62: 'Health Care and Social Assistance',
                 71: 'Arts, Entertainment, and Recreation',
                 72: 'Accommodation and Food Services',
                 81: 'Other Services (except Public Administration)',
                 92: 'Public Administration',
                 99: 'Nonclassifiable Establishments'} 

# Changes Over-Time: Zip-Code

In [ ]:
temp = zbp_detail.groupby(['year', 'zip'])['est'].sum()
temp = temp.unstack()
temp = temp.sort_values(by=2012, axis=1, ascending=False)

counter = 0
for i in np.arange(0, temp.shape[1], 20):
    if counter >= 6:
        break
    ax = temp.iloc[:,i:i+20].plot()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    counter += 1

# Changes Over-Time: Industry

In [ ]:
temp = zbp_detail.groupby(['year', 'naics'])['est'].sum()
ax = temp.unstack().plot()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), labels=[naics_mapping[col] for col in temp.unstack().columns])

# Check Highest Growth Establishment Sizes

In [ ]:
temp = zbp_detail.groupby(['year'])[['n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000']].sum()
ax = temp.plot()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))